In [1]:
import esm
import pickle
import torch
from tqdm import tqdm

c:\Users\jonas\anaconda3\envs\DeepLearningProject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_esm_if1, alphabet = esm.pretrained.esm_if1_gvp4_t16_142M_UR50()
model_esm_if1 = model_esm_if1.eval()

c:\Users\jonas\anaconda3\envs\DeepLearningProject\lib\site-packages\esm\pretrained.py:216: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  "Regression weights not found, predicting contacts will not produce correct results."


In [3]:
def PretrainedEncoder(proteinID):
  fpath = 'C:/Users/jonas/OneDrive/Skrivebord/Deep Learning Project/Data/AlphaFoldDB/' + proteinID + '.pdb' # .pdb format is also acceptable
  chain_ids = ['A']
  structure = esm.inverse_folding.util.load_structure(fpath, chain_ids)

  coords, _ = esm.inverse_folding.multichain_util.extract_coords_from_complex(structure)

  target_chain_id = 'A'     # Always use chain "A", as we only have 1 protein (chain)
  rep = esm.inverse_folding.multichain_util.get_encoder_output_for_complex(model_esm_if1, alphabet, coords, target_chain_id)
  return rep.detach()

In [5]:
PretrainedEncoder("P10384")

tensor([[ 0.0168,  0.1001, -0.2247,  ...,  0.4565,  0.1462, -0.4621],
        [-1.4382, -0.7791, -0.8029,  ...,  0.3053,  0.0196,  0.1065],
        [-1.3222, -0.6228, -0.8992,  ...,  0.2575, -0.0287,  0.0280],
        ...,
        [ 0.1674,  0.8543, -0.0596,  ..., -0.2779, -0.1158, -0.2841],
        [ 0.6398,  0.0417,  0.3769,  ...,  0.2122, -0.2094, -0.3391],
        [ 1.0579, -0.1103,  0.7357,  ...,  0.2159,  0.3422,  0.1547]])

In [33]:
# Was created from google colab
with open('ProteinsUsed.txt', 'rb') as fp:
    ProteinIDs = pickle.load(fp)

In [30]:
# EncodedProteins = []
# with open('C:/Users/jonas/OneDrive/Skrivebord/Deep Learning Project/EncodedProteins.txt', 'wb') as fp:
#     pickle.dump(EncodedProteins,fp)

In [47]:
with open('C:/Users/jonas/OneDrive/Skrivebord/Deep Learning Project/Data/EncodedProteins.txt', 'rb') as fp:
    EncodedProteins = pickle.load(fp)

EncodedProteins.__len__()

3432

In [45]:
for proteinID in tqdm(ProteinIDs):
    if not proteinID in EncodedProteins:
        encoded = PretrainedEncoder(proteinID)
        torch.save(encoded,"C:/Users/jonas/OneDrive/Skrivebord/Deep Learning Project/Data/AlphaFoldDBEncoded/" + proteinID + ".pt")
        EncodedProteins += [proteinID]

100%|██████████| 3432/3432 [11:47:01<00:00, 12.36s/it]  


In [46]:
# Save progress of encoded proteins
with open('C:/Users/jonas/OneDrive/Skrivebord/Deep Learning Project/Data/EncodedProteins.txt', 'wb') as fp:
    pickle.dump(EncodedProteins,fp)